# Imports

In [1]:
import requests as rq
import pandas as pd
import os
import numpy as np
import json
from dotenv import load_dotenv
from sqlalchemy import create_engine
import copy

In [2]:
def show_cols(dict_):
    for cat in dict_.keys():
        print(f'\n\nCategory: {cat}')
        print(*[ i for i in dict_[cat].columns], sep = '\n')

# Definitions

In [3]:
base_urls = {
    "films": "https://swapi.dev/api/films/",
    "people": "https://swapi.dev/api/people/",
    "planets": "https://swapi.dev/api/planets/",
    "species": "https://swapi.dev/api/species/",
    "starships": "https://swapi.dev/api/starships/",
    "vehicles": "https://swapi.dev/api/vehicles/"
}

In [4]:
categories = list(base_urls.keys())
categories

['films', 'people', 'planets', 'species', 'starships', 'vehicles']

Each category has different fields that contain information in the form of an url. I will extract the page id from those fields for each category. 

In [5]:
fields = {
    'films' : ["characters", "planets", "starships", "vehicles", "species"],
    'people' : ["homeworld", "films", "species", "vehicles", "starships"],
    'planets' : ['residents', 'films'],
    'species' : ['people', 'films', 'homeworld'],
    'vehicles' : ['pilots', 'films'],
    'starships' : ['pilots', 'films']
    }

# Consume the API

In [6]:
def scrape_category(url):

    # skip the next url for the first page of pager
    next = url    
    items_list = []

    while next:
        # get the content of the url
        response = rq.get(next)

        # success
        if response.status_code == 200:
            content = response.json()
        elif response.status_code == 404:
            print(f'{url} not found!')
            return
        
        next = content['next']
        items = content['results']

        for item in items:
                    
            # remove created and edited fields
            try:
                del(item['created'])
                del(item['edited'])
            except:
                pass
            
            items_list.append(item)            

    return items_list

## Scrape all the categories and store in *starwars_raw.json*

In [7]:
if not os.path.exists('./data/starwars_raw.json'):
    raw_dict = {cat : scrape_category(base_urls[cat]) for cat in categories}
    
    # store into a json file
    with open('./data/starwars_raw.json', 'w') as file:
        json.dump(raw_dict, file, indent=4 )
    print('Content from Star Wars API stored in a json file!')

else:
    print('The content already exists in a json file!')
    with open('starwars_scrapped.json', 'r') as file:
        raw_dict = json.load(file)

KeyboardInterrupt: 

Function to process the information of an item from a category.
Ex. one character, one planet or one film.

In [ ]:
def process_item(item, fields):

    # create a copy of the item dictionary
    item = copy.deepcopy(item)

    # parse the links from starships, vehicles and species
    for field in fields:
        id_values = []
                        
        if item[field]:  # if the field is not empty
            
            # parse the homeworld (just a single string value)
            if field == 'homeworld':
                # get the homeworld id
                item['homeworld'] = int(item['homeworld'].split('/')[-2])

            else:
                for link in item[field]:

                    # parse the id value in the link    
                    id_values.append(int(link.split('/')[-2]))

                # add the id values into the corresponding field key
                # convert list into tuple, as tuples are hashable
                item[field] = tuple(id_values)

                # each character belongs to only 1 species
                # that way, the species contains
        
        # if species is an empty list, it's a human character
        elif field == 'species' and not item['species']:
            item[field] = 1
                    
        # field has no values (empty list)
        else:
            item[field] = ()
    
    # add the id, extracted from the url
    item['id'] = item['url'].split('/')[-2]
    
    # remove created and edited fields
    try:
        del(item['created'])
        del(item['edited'])
    except:
        pass

    return item

# Store the processed data

In [ ]:
if not os.path.exists('./data/starwars_processed_items.json'):
    # dictionary to store the processed categories
    processed_dict = {}
    
    # process each item for all the categories
    for k,v in raw_dict.items():
        items_processed = []
        for item in v:
            try:
                items_processed.append(process_item(item, fields[k]))
                processed_dict[k] = items_processed
            except:
                print(f'Error in {k}')
    
    # store the information in a json file
    with open('./data/starwars_processed_items.json', 'w') as file:
        json.dump(processed_dict, file, indent = 4)

# the file already exists, so load it
else:
    with open('./data/starwars_processed_items.json', 'r') as file:
        processed_dict= json.load(file)

        # convert lists into tuples after reading from json file
        for cat in categories:
            for item in processed_dict[cat]:
                for field in fields[cat]:
                    try:
                        item[field] = tuple(item[field])
                    # the content of the field is an integer
                    # and not a list. Cannot create a tuple from
                    # an integer using tuple(int)
                    except:
                        pass  # do not convert into a tuple, leave it as integer

    print('Processed data already existed, so the *categories_dict_processed* dictionary will be created from json file.')

In [ ]:
processed_dict['people']

[{'name': 'Luke Skywalker',
  'height': '172',
  'mass': '77',
  'hair_color': 'blond',
  'skin_color': 'fair',
  'eye_color': 'blue',
  'birth_year': '19BBY',
  'gender': 'male',
  'homeworld': 1,
  'films': (1, 2, 3, 6),
  'species': 1,
  'vehicles': (14, 30),
  'starships': (12, 22),
  'url': 'https://swapi.dev/api/people/1/'},
 {'name': 'C-3PO',
  'height': '167',
  'mass': '75',
  'hair_color': 'n/a',
  'skin_color': 'gold',
  'eye_color': 'yellow',
  'birth_year': '112BBY',
  'gender': 'n/a',
  'homeworld': 1,
  'films': (1, 2, 3, 4, 5, 6),
  'species': (2,),
  'vehicles': (),
  'starships': (),
  'url': 'https://swapi.dev/api/people/2/'},
 {'name': 'R2-D2',
  'height': '96',
  'mass': '32',
  'hair_color': 'n/a',
  'skin_color': 'white, blue',
  'eye_color': 'red',
  'birth_year': '33BBY',
  'gender': 'n/a',
  'homeworld': 8,
  'films': (1, 2, 3, 4, 5, 6),
  'species': (2,),
  'vehicles': (),
  'starships': (),
  'url': 'https://swapi.dev/api/people/3/'},
 {'name': 'Darth Vader'

# Dataframes

## Create the dataframes
Create a dictionary to store the dataframes from each category

In [ ]:
dataframes = dict.fromkeys(categories)

In [ ]:
for cat in categories:
    df = pd.DataFrame(processed_dict[cat])
    #df['id'] = df.index + 1

    # rename columns to add '_id' to the "fields"
    rename_dict = {field : f'{field}_id' for field in fields[cat]}
    rename_dict.update({'id' : f'{cat}_id'})
    df.rename(columns = rename_dict, inplace = True)

    # reorder the columns to place id in first place
    all_columns_but_cat_id = [col for col in df.columns if col != f'{cat}_id']
    sorted_columns = [f'{cat}_id'] + all_columns_but_cat_id
    dataframes[cat] = df[sorted_columns]
#df.to_csv('./data/starwars_characters.csv', index = False)

## Rename some columns
Make some renamings to the column names

In [ ]:
col_rename_dict= {
    
    'films': {
        'characters_id': 'character_id',
        'films_id': 'film_id',
        'planets_id': 'planet_id',
        'episode_id': 'episode',
        'starships_id': 'starship_id',
        'vehicles_id': 'vehicle_id',
    },
    
    'people': {
        'people_id': 'character_id',
        'films_id': 'film_id',
        'vehicles_id': 'vehicle_id',
        'starships_id': 'starship_id'
    },

    'planets': {
        'planets_id': 'planet_id',
        'films_id': 'film_id'
    },

    'species': {
        'people_id': 'character_id',
        'films_id': 'film_id'
    },

    'vehicles': {
        'pilots_id': 'pilot_id',
        'vehicles_id': 'vehicle_id',
        'films_id': 'film_id'
    },

    'starships': {
        'pilots_id': 'pilot_id',
        'starships_id': 'starship_id',
        'films_id': 'film_id'
    }
}

In [ ]:
for cat in categories:
    dataframes[cat].rename(columns=col_rename_dict[cat], inplace=True)

## Clean the datasets

### Clean people

In [ ]:
df = dataframes['people']
df.sample(5)

,character_id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld_id,film_id,species_id,vehicle_id,starship_id,url
23,24,Lando Calrissian,177,79,black,dark,brown,31BBY,male,30,"(2, 3)",1,(),"(10,)",https://swapi.dev/api/people/25/
77,78,Grievous,216,159,none,"brown, white","green, yellow",unknown,male,59,"(6,)","(36,)","(60,)","(74,)",https://swapi.dev/api/people/79/
74,75,Wat Tambor,193,48,none,"green, grey",unknown,unknown,male,56,"(5,)","(33,)",(),(),https://swapi.dev/api/people/76/
10,11,Anakin Skywalker,188,84,blond,fair,blue,41.9BBY,male,1,"(4, 5, 6)",1,"(44, 46)","(39, 59, 65)",https://swapi.dev/api/people/11/
22,23,Bossk,190,113,none,green,red,53BBY,male,29,"(2,)","(7,)",(),(),https://swapi.dev/api/people/24/


In [ ]:
df.dtypes

character_id     int64
name            object
height          object
mass            object
hair_color      object
skin_color      object
eye_color       object
birth_year      object
gender          object
homeworld_id     int64
film_id         object
species_id      object
vehicle_id      object
starship_id     object
url             object
dtype: object

In [ ]:
df.mass = df.mass.replace('unknown', np.nan)
df.mass = df.mass.str.replace(',', '', regex=False)
df.mass = df.mass.astype('float') 

In [ ]:
df.height = df.height.replace('unknown', np.nan).astype('float')

In [ ]:
df.hair_color = df.hair_color.replace('n/a', np.nan)

In [ ]:
df.birth_year = df.birth_year.str.replace('BBY', ' BBY')

### Clean films

In [ ]:
df = dataframes['films']
df.sample(5)

,film_id,title,episode,opening_crawl,director,producer,release_date,character_id,planet_id,starship_id,vehicle_id,species_id,url
4,5,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,"(2, 3, 6, 7, 10, 11, 20, 21, 22, 33, 35, 36, 4...","(1, 8, 9, 10, 11)","(21, 32, 39, 43, 47, 48, 49, 52, 58)","(4, 44, 45, 46, 50, 51, 53, 54, 55, 56, 57)","(1, 2, 6, 12, 13, 15, 28, 29, 30, 31, 32, 33, ...",https://swapi.dev/api/films/5/
5,6,Revenge of the Sith,3,War! The Republic is crumbling\r\nunder attack...,George Lucas,Rick McCallum,2005-05-19,"(1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 20, 21, ...","(1, 2, 5, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19)","(2, 32, 48, 59, 61, 63, 64, 65, 66, 68, 74, 75)","(33, 50, 53, 56, 60, 62, 67, 69, 70, 71, 72, 7...","(1, 2, 3, 6, 15, 19, 20, 23, 24, 25, 26, 27, 2...",https://swapi.dev/api/films/6/
3,4,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,"(2, 3, 10, 11, 16, 20, 21, 32, 33, 34, 35, 36,...","(1, 8, 9)","(31, 32, 39, 40, 41)","(33, 34, 35, 36, 37, 38, 42)","(1, 2, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, ...",https://swapi.dev/api/films/4/
1,2,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"(1, 2, 3, 4, 5, 10, 13, 14, 18, 20, 21, 22, 23...","(4, 5, 6, 27)","(3, 10, 11, 12, 15, 17, 21, 22, 23)","(8, 14, 16, 18, 19, 20)","(1, 2, 3, 6, 7)",https://swapi.dev/api/films/2/
0,1,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","(1, 2, 3)","(2, 3, 5, 9, 10, 11, 12, 13)","(4, 6, 7, 8)","(1, 2, 3, 4, 5)",https://swapi.dev/api/films/1/


In [ ]:
df.species_id.unique()

array([(1, 2, 3, 4, 5), (1, 2, 3, 6, 7), (1, 2, 3, 5, 6, 8, 9, 10, 15),
       (1, 2, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27),
       (1, 2, 6, 12, 13, 15, 28, 29, 30, 31, 32, 33, 34, 35),
       (1, 2, 3, 6, 15, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34, 35, 36, 37)],
      dtype=object)

In [ ]:
df.dtypes

film_id           int64
title            object
episode           int64
opening_crawl    object
director         object
producer         object
release_date     object
character_id     object
planet_id        object
starship_id      object
vehicle_id       object
species_id       object
url              object
dtype: object

In [ ]:
df.release_date = pd.to_datetime(df.release_date)

### Clean planets

In [ ]:
df = dataframes['planets']
df.sample(5)

,planet_id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents_id,film_id,url
25,26,Bestine IV,26,680,6400,temperate,unknown,"rocky islands, oceans",98,62000000,"(19,)",(),https://swapi.dev/api/planets/26/
21,22,Corellia,25,329,11000,temperate,1 standard,"plains, urban, hills, forests",70,3000000000,"(14, 18)",(),https://swapi.dev/api/planets/22/
22,23,Rodia,29,305,7549,hot,1 standard,"jungles, oceans, urban, swamps",60,1300000000,"(15,)",(),https://swapi.dev/api/planets/23/
51,52,Serenno,unknown,unknown,unknown,unknown,unknown,"rainforests, rivers, mountains",unknown,unknown,"(67,)",(),https://swapi.dev/api/planets/52/
8,9,Coruscant,24,368,12240,temperate,1 standard,"cityscape, mountains",unknown,1000000000000,"(34, 55, 74)","(3, 4, 5, 6)",https://swapi.dev/api/planets/9/


In [ ]:
df.dtypes

planet_id           int64
name               object
rotation_period    object
orbital_period     object
diameter           object
climate            object
gravity            object
terrain            object
surface_water      object
population         object
residents_id       object
film_id            object
url                object
dtype: object

In [ ]:
for col in ['rotation_period', 'orbital_period', 'diameter', 'surface_water', 'population']:
    df[col] = df[col].replace('unknown', np.nan).astype('float')

In [ ]:
df.population = df.population / 1E6
df.rename(columns = {'population' : 'population_millions'}, inplace = True)

In [ ]:
df.gravity.unique()

array(['1 standard', '1.1 standard', 'N/A',
       '1.5 (surface), 1 standard (Cloud City)', '0.85 standard',
       '0.9 standard', '0.56 standard', '0.75 standard', 'unknown',
       '0.62 standard', '1', '1.56', '0.9', '0.98'], dtype=object)

In [ ]:
df.gravity = df.gravity.str.replace(' standard', '').str.replace(df.gravity[5], '1.5')
df.gravity = df.gravity.replace('unknown', np.nan)

### Clean species

In [ ]:
df = dataframes['species']
df.sample(5)

,species_id,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld_id,language,character_id,film_id,url
18,19,Toong,unknown,sentient,200,"grey, green, yellow",none,orange,unknown,41,Tundan,"(50,)","(4, 6)",https://swapi.dev/api/species/19/
35,36,Kaleesh,reptile,sentient,170,"brown, orange, tan",none,yellow,80,59,Kaleesh,"(79,)","(6,)",https://swapi.dev/api/species/36/
3,4,Rodian,sentient,reptilian,170,"green, blue",n/a,black,unknown,23,Galatic Basic,"(15,)","(1,)",https://swapi.dev/api/species/4/
20,21,Nautolan,amphibian,sentient,180,"green, blue, brown, red",none,black,70,44,Nautila,"(53,)","(4,)",https://swapi.dev/api/species/21/
15,16,Aleena,reptile,sentient,80,"blue, gray",none,unknown,79,38,Aleena,"(47,)","(4,)",https://swapi.dev/api/species/16/


In [ ]:
df.dtypes

species_id           int64
name                object
classification      object
designation         object
average_height      object
skin_colors         object
hair_colors         object
eye_colors          object
average_lifespan    object
homeworld_id        object
language            object
character_id        object
film_id             object
url                 object
dtype: object

In [ ]:
df.average_height = df.average_height.replace('unknown', np.nan).replace('n/a', np.nan).astype('float')

In [ ]:
df.average_lifespan = df.average_lifespan.replace('unknown', np.nan).replace('indefinite', 9999).astype('float')

In [ ]:
df.loc[1, 'homeworld_id'] = np.nan
df.homeworld_id = df.homeworld_id.astype('float')

### Clean vehicles

In [ ]:
df = dataframes['vehicles']
df.sample(5)

,vehicle_id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilot_id,film_id,url
0,1,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,(),"(1, 5)",https://swapi.dev/api/vehicles/4/
15,16,Armored Assault Tank,Armoured Assault Tank,Baktoid Armor Workshop,unknown,9.75,55,4,6,unknown,unknown,repulsorcraft,(),"(4,)",https://swapi.dev/api/vehicles/35/
11,12,TIE/IN interceptor,Twin Ion Engine Interceptor,Sienar Fleet Systems,unknown,9.6,1250,1,0,75,2 days,starfighter,(),"(3,)",https://swapi.dev/api/vehicles/26/
10,11,Bantha-II cargo skiff,Bantha-II,Ubrikkian Industries,8000,9.5,250,5,16,135000,1 day,repulsorcraft cargo skiff,(),"(3,)",https://swapi.dev/api/vehicles/25/
30,31,Tsmeu-6 personal wheel bike,Tsmeu-6 personal wheel bike,Z-Gomot Ternbuell Guppat Corporation,15000,3.5,330,1,1,10,none,wheeled walker,"(79,)","(6,)",https://swapi.dev/api/vehicles/60/


In [ ]:
df.cost_in_credits = df.cost_in_credits.replace('unknown', np.nan)
df.cost_in_credits = df.cost_in_credits.astype('float')

In [ ]:
for col in ['max_atmosphering_speed' ,'crew', 'passengers', 'cargo_capacity']:
    df[col] = df[col].replace('unknown', np.nan).replace('none', np.nan)
    df[col] = df[col].astype('float')

In [ ]:
df.length = df.length.replace('unknown', np.nan)
df.length = df.length.astype('float')

In [ ]:
df.consumables = df.consumables.replace('0', 'none')

### Clean starships

In [ ]:
df = dataframes['starships']
df.sample(5)

,starship_id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilot_id,film_id,url
22,23,AA-9 Coruscant freighter,Botajef AA-9 Freighter-Liner,Botajef Shipyards,unknown,390,unknown,unknown,30000,unknown,unknown,unknown,unknown,freighter,(),"(5,)",https://swapi.dev/api/starships/47/
35,36,V-wing,Alpha-3 Nimbus-class V-wing starfighter,Kuat Systems Engineering,102500,7.9,1050,1,0,60,15 hours,1.0,unknown,starfighter,(),"(6,)",https://swapi.dev/api/starships/75/
0,1,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,(),"(1, 3, 6)",https://swapi.dev/api/starships/2/
14,15,A-wing,RZ-1 A-wing Interceptor,"Alliance Underground Engineering, Incom Corpor...",175000,9.6,1300,1,0,40,1 week,1.0,120,Starfighter,"(29,)","(3,)",https://swapi.dev/api/starships/28/
25,26,Republic Assault ship,Acclamator I-class assault ship,Rothana Heavy Engineering,unknown,752,unknown,700,16000,11250000,2 years,0.6,unknown,assault ship,(),"(5,)",https://swapi.dev/api/starships/52/


In [ ]:
df.loc[0, 'crew'] = 165

In [ ]:
for col in ['cost_in_credits', 'length', 'max_atmosphering_speed', 'crew', 'passengers', 'cargo_capacity', 'hyperdrive_rating', 'MGLT']:
    try:
        df[col] = df[col].replace('unknown', np.nan).replace('none', np.nan).replace('n/a', np.nan)
        df[col] = df[col].str.replace(',', '', regex = False).str.replace('km', '')
        #df[col] = df[col].astype('float')
    except Exception as e:
        print(f'Error in {col}: {e}')

In [ ]:
for col in ['cost_in_credits', 'length', 'max_atmosphering_speed', 'crew', 'passengers', 'cargo_capacity', 'hyperdrive_rating', 'MGLT']:
    try:
        df[col] = df[col].astype('float')
    except:
        print(f'error with {col}')

## Export clean datasets into csv files

In [ ]:
data_path = './data'
for cat in categories:
    filename = f'{cat}_dataframe.csv'
    if os.path.exists(f'{data_path}/csv/{filename}'):
        print(f'File {filename} already exist!')
        pass
    else:
        os.makedirs(f'{data_path}/csv/', exist_ok=True)
        df = dataframes[cat]
        df.to_csv(f'{data_path}/csv/{cat}_dataframe.csv', index = False)
print(f'Dataframes of each normalized category are stored in {data_path}/csv/ as csv files!')

Dataframes of each normalized category are stored in ./data/csv/ as csv files!


# Junction tables

(many-to-many relationships in the database)

1. **films_people**: Links films to the characters that appeared in them.

    - character_id: Foreign Key referencing the `people` table.
    - film_id: Foreign Key referencing the films table.

2. **films_planets**: Links films to the planets that appeared in them.

    - planet_id: Foreign Key referencing the `planets` table.
    - film_id: Foreign Key referencing the `films` table.

3. **films_starships**: Links films to the starships that appeared in them.

    - starship_id: Foreign Key referencing the `starships` table.
    - film_id: Foreign Key referencing the `films` table.

4. **films_vehicles**: Links films to the vehicles that appeared in them.

    - vehicle_id: Foreign Key referencing the `vehicles` table.
    - film_id: Foreign Key referencing the `films` table.

5. **films_species**: Links films to the species that appeared in them.

    - species_id: Foreign Key referencing the `species` table.
    - film_id: Foreign Key referencing the `films` table.

6. **people_starships**: Links people (pilots) to the starships they have piloted.

    - character_id: Foreign Key referencing the `people` table.
    - starship_id: Foreign Key referencing the `starships` table.

7. **people_vehicles**: Links people (pilots) to the vehicles they have piloted.

    - character_id: Foreign Key referencing the `people` table.
    - vehicle_id: Foreign Key referencing the vehicles table.

In [ ]:
junction_tables = [
    'people_vehicles', 
    'people_starships',
    
    'films_people',
    'films_planets',
    'films_starships',
    'films_vehicles',
    'films_species'
    ]

junction_tables_dict = {i:None for i in junction_tables}

## Junction tables for people:

In [ ]:
columns = ['vehicle_id', 'starship_id']
data = dataframes['people'].loc[:, columns + ['character_id']]

for col in columns:
    table_name = f'people_{col}'.replace('_id', 's')
    junction_tables_dict[table_name] = (data.loc[:, ['character_id', col]]
                                        .explode(col)
                                        .explode('character_id')
                                        .dropna()
                                        .reset_index(drop = True)
                                        .sort_values('character_id')
                                        )

## Junction tables for films:

In [ ]:
columns = ['character_id', 'species_id', 'planet_id', 'vehicle_id', 'starship_id']
data = dataframes['films'].loc[:, columns + ['film_id']]

for col in columns:
    table_name = f'films_{col}'
    if col == 'species_id':
        table_name = table_name.replace('_id', '')
    elif col == 'character_id':
        table_name = 'films_people'
    else:
        table_name = table_name.replace('_id', 's')
        
    junction_tables_dict[table_name] = (data.loc[:, ['film_id', col ]]
                                        .explode(col)
                                        .reset_index(drop = True)
                                        .sort_values('film_id')
    )

# Normalization
Next step is to normalize the datasets in order to create the database.

In [ ]:
dataframes_normalized = copy.deepcopy(dataframes)

In [ ]:
columns_to_drop = {
    'films' : ['character_id', 'planet_id', 'species_id', 'vehicle_id', 'starship_id', 'url'],
    'people' : ['film_id', 'species_id', 'vehicle_id', 'starship_id', 'url'],
    'planets' : ['residents_id', 'film_id', 'url'],
    'species' : ['character_id', 'film_id', 'url'],
    'starships' : ['pilot_id', 'film_id', 'url'],
    'vehicles' : ['pilot_id', 'film_id', 'url'],
}  

### Drop the corresponding columns in order to normalize the tables

In [ ]:
for cat in dataframes_normalized.keys():
    dataframes_normalized[cat].drop(columns_to_drop[cat], axis='columns', inplace = True)

## Store the normalized dataframes

In [ ]:
data_path = './data'
for cat in categories:
    filename = f'{cat}_dataframe.csv'
    if os.path.exists(f'{data_path}/csv_normalized/{filename}'):
        print(f'File {filename} already exist!')
        pass
    else:
        os.makedirs(f'{data_path}/csv_normalized/', exist_ok=True)
        df = dataframes_normalized[cat]
        df.to_csv(f'{data_path}/csv_normalized/{cat}_dataframe_normalized.csv', index = False)
print(f'Dataframes of each normalized category are stored in {data_path}/csv_normalized/ as csv files!')

Dataframes of each normalized category are stored in ./data/csv_normalized/ as csv files!


# Insert data into the database

## Load database parameters from `.env` file

In [ ]:
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

## Create the db connection

In [ ]:
connection_string = (
    f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
)

# --- 4. Create the SQLAlchemy Engine ---
try:
    engine = create_engine(connection_string)
    print("SQLAlchemy Engine created successfully. 🛠️")
except Exception as e:
    print(f"Error creating engine: {e}")

SQLAlchemy Engine created successfully. 🛠️


## Populate the data into the database

The order of tables to be filled must be:
1. planets
2. species
3. starships
4. vehicles
5. films
6. people

In [ ]:
categories_sorted = ['planets', 'species', 'vehicles', 'starships', 'films', 'people']
insert = False

### Insert the category tables into the database

In [ ]:
def insert_category(cat, dictionary):
    df = dictionary[cat]
    try:
        df.to_sql(name=cat, con=engine, if_exists='append', index=False)
        print(f"DataFrame for category '{cat}' inserted successfully into the database. ✅\n")
    except Exception as e:
        print(f"\\ Error inserting DataFrame for category '{cat}': \n{e}\n\n")

In [ ]:
dataframes_normalized['planets'].head()

,planet_id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population_millions
0,1,Tatooine,23.0,304.0,10465.0,arid,1,desert,1.0,0.200
1,2,Alderaan,24.0,364.0,12500.0,temperate,1,"grasslands, mountains",40.0,2000.000
2,3,Yavin IV,24.0,4818.0,10200.0,"temperate, tropical",1,"jungle, rainforests",8.0,0.001
3,4,Hoth,23.0,549.0,7200.0,frozen,1.1,"tundra, ice caves, mountain ranges",100.0,NaN
4,5,Dagobah,23.0,341.0,8900.0,murky,N/A,"swamp, jungles",8.0,NaN


In [ ]:
for cat in categories_sorted:
    query = f'select * from {cat} limit 1 ;'
    # if table is empty, fill it with the corresponding data
    if pd.read_sql(query, con = engine).shape[0] == 0:
        insert_category(cat, dataframes_normalized)
    else:
        print(f'{cat} table already exists in database!')

DataFrame for category 'planets' inserted successfully into the database. ✅

DataFrame for category 'species' inserted successfully into the database. ✅

DataFrame for category 'vehicles' inserted successfully into the database. ✅

DataFrame for category 'starships' inserted successfully into the database. ✅

DataFrame for category 'films' inserted successfully into the database. ✅

DataFrame for category 'people' inserted successfully into the database. ✅



### Insert junction tables

In [ ]:
print(junction_tables_dict['people_vehicles'].head())

   character_id vehicle_id
0             1         14
1             1         30
2             5         30
3            10         38
4            11         44


In [ ]:
for table in junction_tables_dict.keys():
    query = f'select * from {table} limit 1 ;'
    # if table is empty, fill it with the corresponding data
    if pd.read_sql(query, con = engine).shape[0] == 0:
        insert_category(table, junction_tables_dict)
    else:
        print(f'{cat} table already exists in database!') 

\ Error inserting DataFrame for category 'people_vehicles': 
(pymysql.err.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`starwars_db`.`people_vehicles`, CONSTRAINT `people_vehicles_ibfk_2` FOREIGN KEY (`vehicle_id`) REFERENCES `vehicles` (`vehicle_id`))')
[SQL: INSERT INTO people_vehicles (character_id, vehicle_id) VALUES (%(character_id)s, %(vehicle_id)s)]
[parameters: [{'character_id': 1, 'vehicle_id': 14}, {'character_id': 1, 'vehicle_id': 30}, {'character_id': 5, 'vehicle_id': 30}, {'character_id': 10, 'vehicle_id': 38}, {'character_id': 11, 'vehicle_id': 44}, {'character_id': 11, 'vehicle_id': 46}, {'character_id': 13, 'vehicle_id': 19}, {'character_id': 17, 'vehicle_id': 14}  ... displaying 10 of 13 total bound parameter sets ...  {'character_id': 69, 'vehicle_id': 45}, {'character_id': 78, 'vehicle_id': 60}]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


\ Error inserting DataFrame for category 'people_starships': 
(py